# Image Rotation Angle Prediction Model

This notebook implements an image rotation angle prediction model using deep learning techniques. The model is trained on a dataset of images and their corresponding rotation angles.

## Dependencies

The following libraries are required to run the code successfully:

- pandas: Used for data manipulation and analysis.
- numpy: Used for mathematical operations on arrays.
- tensorflow.keras.models.Sequential: The Keras Sequential model class for building the neural network model.
- tensorflow.keras.layers: Contains various layers for constructing the neural network architecture.
- tensorflow.keras.preprocessing.image.ImageDataGenerator: Used for real-time data augmentation and preprocessing of images.
- sklearn.model_selection.train_test_split: Used for splitting the dataset into training and validation sets.

Make sure these libraries are installed before running the code.

## Dataset

The dataset used for training and validation is stored in a CSV file named 'augmented_data.csv'. The CSV file should contain two columns:

- 'Image Address': Contains the file paths or URLs of the images.
- 'Rotation Angle': Contains the rotation angles in degrees for each image.

## Loading and Preprocessing the Dataset

The code begins by importing the necessary libraries and loading the dataset using pandas. It then performs some preprocessing steps on the dataset:

- Scaling the rotation angles: The rotation angles are scaled by adding 90 degrees and dividing by 5. This step ensures that the angles are within a suitable range for training the model.
- Modifying image paths: The image paths are modified to replace backslashes with forward slashes to ensure compatibility with different operating systems.

## ImageDataGenerator

An ImageDataGenerator is created to perform real-time data augmentation and preprocessing. The generator is configured with the following parameters:

- rescale: Normalizes the pixel values of the images between 0 and 1.
- validation_split: Splits the data into training and validation sets based on the specified percentage.

## Data Generators

Two data generators are created using the ImageDataGenerator:

- train_generator: Generates training data by flowing from the dataframe. It uses the training subset of the data.
- validation_generator: Generates validation data by flowing from the dataframe. It uses the validation subset of the data.

These generators automatically load images, preprocess them, and generate batches of images and their corresponding rotation angles for training and validation.

## Neural Network Model

The code defines a sequential neural network model using the Keras Sequential class. The model architecture consists of the following layers:

- Conv2D: Applies convolutional filters to extract features from input images.
- MaxPooling2D: Performs max pooling to downsample the feature maps.
- Flatten: Flattens the feature maps into a 1D vector.
- Dense: Fully connected layers for classification.
- Activation: Activation functions applied to the outputs of the previous layers.

The final layer uses a softmax activation function to predict the rotation angle from the available classes.

## Training the Model

The model is compiled with the Adam optimizer and the sparse categorical cross-entropy loss function. It is then trained using the `fit` function with the training generator. The number of epochs can be modified as needed.

## Evaluating the Model

The trained model is evaluated using the validation generator. The `evaluate` function returns the loss and accuracy metrics. These metrics indicate the performance of the model on the validation set.

## Conclusion

This notebook demonstrates how to build and train a deep learning model for predicting rotation angles of images. By leveraging the power of convolutional neural networks and data augmentation, accurate predictions can be made for various rotation angles. The trained model can be further optimized or fine-tuned to improve performance on specific tasks.


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
# Load the dataset
data = pd.read_csv('./augmented_data.csv')
data['Rotation Angle'] = (data['Rotation Angle'] + 90)/5
data['Image Address'] = data['Image Address'].str.replace("\\", "/")

In [3]:
# Extract the image paths and rotation angles
image_paths = data['Image Address'].tolist()
rotation_angles = data['Rotation Angle'].tolist()

# Define the desired image size
image_width, image_height = 180, 240


In [4]:
# Convert rotation angles to integer values
rotation_angles = [int(angle) for angle in rotation_angles]

In [5]:
# Create an ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values between 0 and 1
    validation_split=0.2
)

In [6]:
# Create the data generator for training set
train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='Image Address',
    y_col='Rotation Angle',
    target_size=(image_width, image_height),
    batch_size=32,
    subset='training',
    shuffle=True,
    class_mode='raw'  # Use 'raw' for integer labels
)


Found 39250 validated image filenames.


In [7]:
# Create the data generator for validation set
validation_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='Image Address',
    y_col='Rotation Angle',
    target_size=(image_width, image_height),
    batch_size=32,
    subset='validation',
    shuffle=True,
    class_mode='raw'  # Use 'raw' for integer labels
)

Found 9812 validated image filenames.


In [8]:
# Create the neural network model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(37, activation='softmax'))  # Output layer with 37 classes for 37 possible angles


In [9]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
# Train the model
model.fit(train_generator, epochs=5, validation_data=validation_generator)



Epoch 1/5
1227/1227 [==============================] - 1762s 1s/step - loss: 0.0801 - accuracy: 0.9779 - val_loss: 0.0568 - val_accuracy: 0.9842
Epoch 2/5
1227/1227 [==============================] - 1719s 1s/step - loss: 0.0204 - accuracy: 0.9926 - val_loss: 0.1047 - val_accuracy: 0.9801
Epoch 3/5
1227/1227 [==============================] - 1733s 1s/step - loss: 0.0124 - accuracy: 0.9968 - val_loss: 0.1738 - val_accuracy: 0.9732
Epoch 4/5
1227/1227 [==============================] - 1711s 1s/step - loss: 0.0072 - accuracy: 0.9978 - val_loss: 0.1119 - val_accuracy: 0.9820
Epoch 5/5
1227/1227 [==============================] - 1724s 1s/step - loss: 0.0034 - accuracy: 0.9993 - val_loss: 0.1785 - val_accuracy: 0.9746


In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

307/307 [==============================] - 98s 320ms/step - loss: 0.1785 - accuracy: 0.9746
Validation Loss: 0.17854377627372742
Validation Accuracy: 0.974622905254364


In [12]:
# Save the weights
model.save_weights('model_weights.h5')